In [1]:
import numpy as np
import pandas as pd 
import os
import random
import time

In [2]:
DEBUG_MODEL=False
INPUT_DIR="/vc_data/users/v-mxiong/qc/QvLabeldata1.txt"
SCHEMA_DIR="/vc_data/users/v-mxiong/qc/QvSchema1.txt"
SEED=42
def read_data_txt(input_file,schema_file,is_debug=DEBUG_MODEL):
    with open(schema_file,"r") as input_object:
        schema_line=input_object.readlines()
    head=schema_line[0].strip().split("\t")
    with open(input_file,"r") as input_object:
        input_lines=input_object.readlines()
    if DEBUG_MODEL==True:
        input_lines=input_lines[:50000]
    input_lines=[ line.strip().split("\t") for line in input_lines]
    input_lines=[ line for line in input_lines if len(line)==len(head)]
    data_rows=input_lines
    input_df=pd.DataFrame(data_rows,columns=head)
    return input_df
def sample_negative(df,negative_rate=1):
    negative_df=df[df.label==0]
    positive_df=df[df.label==1]
    sample_num=len(positive_df)*negative_rate
    
    sample_df=negative_df.sample(n=sample_num,random_state=SEED)
    
    # .sample(frac=1) 用于打乱顺序
    return_df=pd.concat([positive_df,sample_df]).sample(frac=1)
    return_df=return_df.reset_index(drop=True)
    return return_df

In [3]:
input_df=read_data_txt(INPUT_DIR,SCHEMA_DIR)
input_df["label"]=input_df["label"].replace(["0","1"],[0,1])

In [4]:
input_df["label"].value_counts()
sat_rate=input_df["label"].mean()
input_df["ClickCount"]=input_df["ClickCount"].astype(int)
click_rate=(input_df["ClickCount"]>0).mean()
noclick_rate=1-click_rate
show_df=pd.DataFrame({"query_num":[len(input_df)],"click_rate":[click_rate],\
"noclick_rate":[noclick_rate],"sat_rate":[sat_rate],"dsat_rate":[click_rate-sat_rate]})
show_df

# sum_click=sum(input_df["ClickCount"])
# sum_count=sum(input_df["Count"].astype(int))
# sum_click/sum_count

,query_num,click_rate,noclick_rate,sat_rate,dsat_rate
0,975585,0.03581,0.96419,0.025329,0.010481


In [5]:
input_df.columns

Index(['Query', 'No_ClickCount', 'ClickCount', 'SatCount', 'DsatCount',
       'Count', 'Market', 'QueryVector', 'label'],
      dtype='object')

In [6]:
input_df=sample_negative(input_df)

In [7]:
input_df["label"].value_counts()

1    24711
0    24711
Name: label, dtype: int64

In [8]:
import tqdm
data,label=[],[]
df=input_df
label=input_df["label"].values.tolist()
data=input_df["QueryVector"].values.tolist()
data=[ vector.split("|") for vector in data]
data=[ [int(v) for v in vector] for vector in data ]    


In [9]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test=train_test_split(data,label,test_size=0.2,random_state=42)


In [15]:
from sklearn.neighbors import KNeighborsClassifier #
from sklearn import linear_model

from sklearn.metrics import classification_report
lg =linear_model.LogisticRegression(C=0.01)  # 默认使用L2正则化避免过拟合，C=1.0表示正则力度
lg.fit(X_train, Y_train)
       
y_predict = lg.predict(X_test)
print("训练集准确率：", lg.score(X_train, Y_train))
print("准确率：", lg.score(X_test, Y_test))
print("召回率：", classification_report(Y_test, y_predict, labels=[0, 1], target_names=["不满意", "满意"]))

训练集准确率： 0.5715405822394213
准确率： 0.5689428426909459
召回率：               precision    recall  f1-score   support

         不满意       0.56      0.58      0.57      4905
          满意       0.57      0.56      0.57      4980

    accuracy                           0.57      9885
   macro avg       0.57      0.57      0.57      9885
weighted avg       0.57      0.57      0.57      9885



In [19]:
print("训练集准确率：", knn.score(X_train, Y_train))

训练集准确率： 0.706882161013734


In [16]:
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier #
knn=KNeighborsClassifier(n_neighbors=5) #选择knn邻近模型，返回一个模型对象
knn.fit(X_train,Y_train) #训练
y_predict=knn.predict(X_test) #预测
acc=knn.score(X_test,Y_test) #计算准确率
print("训练集准确率：", knn.score(X_train, Y_train))
print(acc)
print("召回率：", classification_report(Y_test, y_predict, labels=[0, 1], target_names=["不满意", "满意"]))

训练集准确率： 0.5715405822394213
0.5503287809812848
召回率：               precision    recall  f1-score   support

         不满意       0.55      0.49      0.52      4905
          满意       0.55      0.61      0.58      4980

    accuracy                           0.55      9885
   macro avg       0.55      0.55      0.55      9885
weighted avg       0.55      0.55      0.55      9885



In [18]:
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

param = {'criterion':['gini'],'max_depth':[30,50,60,100],'min_samples_leaf':[2,3,5,10],'min_impurity_decrease':[0.1,0.2,0.5]}
grid = GridSearchCV(DecisionTreeClassifier(criterion="entropy"),param_grid=param,cv=6)
grid.fit(X_train,Y_train)
print('最优分类器:',grid.best_params_,'最优分数:', grid.best_score_)  # 得到最优的参数和分值

最优分类器: {'criterion': 'gini', 'max_depth': 30, 'min_impurity_decrease': 0.1, 'min_samples_leaf': 2} 最优分数: 0.5009484815244795
